<a href="https://colab.research.google.com/github/GregoriCastelhano/SQL_Analysis/blob/main/SQL_ANALYSIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Como a Monte Venêto pode estar bem posicionada para o lançamento do seu primeiro vinho?**

### **Levamos em consideração 3 dimensões principais:**


**Região**

- Quais as regiões apresentam maior produção de vinhos avaliados pela revista?
- Quais estão no Brasil?

**Uva**

- Quais os tipos de uvas são mais bem avaliadas?
- Quais uvas são utilizadas pelas vinicolas nacionais?

**Preço**

- Qual a média de preço dos vinhos?
- Qual o preço médio dos vinhos brasileiros avaliados pela revista?

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 26.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=457a7cc344f1087c5c801f6216b3738aa0cfcb7042d4851f6cda1979cd7a6773
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
from pyspark.sql import*
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [3]:
from pyspark.sql.functions import*

In [4]:
df = spark.read.format("csv") \
     .option("header", "true") \
     .option("inferSchema", "true") \
     .load("/content/drive/MyDrive/Portfolio/Vinho_DataSet/winemag_data.csv")

In [5]:
df.createOrReplaceTempView("winemag_data")

#**Exploração dos dados (EDA)**

In [6]:
result = spark.sql("SELECT * FROM winemag_data \
                    LIMIT 5")
result.show()


+---+-------+--------------------+--------------------+------+-----+--------------+-----------------+-----------------+------------------+--------------------+
|_c0|country|         description|         designation|points|price|      province|         region_1|         region_2|           variety|              winery|
+---+-------+--------------------+--------------------+------+-----+--------------+-----------------+-----------------+------------------+--------------------+
|  0|     US|This tremendous 1...|   Martha's Vineyard|    96|235.0|    California|      Napa Valley|             Napa|Cabernet Sauvignon|               Heitz|
|  1|  Spain|Ripe aromas of fi...|Carodorum Selecci...|    96|110.0|Northern Spain|             Toro|             null|     Tinta de Toro|Bodega Carmen Rod...|
|  2|     US|Mac Watson honors...|Special Selected ...|    96| 90.0|    California|   Knights Valley|           Sonoma|   Sauvignon Blanc|            Macauley|
|  3|     US|This spent 20 mon...|      

**Quais os países e regiões apresentam maior produção de vinhos avaliados pela revista?**



In [7]:
result = spark.sql("select country as pais, count(country) as soma_pais\
                    from winemag_data \
                    group by  country \
                    order by soma_pais desc")
result.show()

+------------+---------+
|        pais|soma_pais|
+------------+---------+
|          US|    62397|
|       Italy|    23478|
|      France|    21098|
|       Spain|     8268|
|       Chile|     5816|
|   Argentina|     5631|
|    Portugal|     5322|
|   Australia|     4957|
| New Zealand|     3320|
|     Austria|     3057|
|     Germany|     2452|
|South Africa|     2258|
|      Greece|      884|
|      Israel|      630|
|     Hungary|      231|
|      Canada|      196|
|     Romania|      139|
|    Slovenia|       94|
|     Uruguay|       92|
|     Croatia|       89|
+------------+---------+
only showing top 20 rows



In [8]:
result = spark.sql("select province as regiao, count(province) as soma_regiao \
                    from winemag_data \
                    group by province \
                    order by soma_regiao desc")
result.show()

+------------------+-----------+
|            regiao|soma_regiao|
+------------------+-----------+
|        California|      44496|
|        Washington|       9749|
|           Tuscany|       7281|
|          Bordeaux|       6111|
|    Northern Spain|       4892|
|  Mendoza Province|       4742|
|            Oregon|       4589|
|          Burgundy|       4306|
|          Piedmont|       4092|
|            Veneto|       3962|
|   South Australia|       2998|
| Sicily & Sardinia|       2544|
|          New York|       2426|
|Northeastern Italy|       1981|
|      Loire Valley|       1784|
|            Alsace|       1680|
|       Marlborough|       1655|
|  Southwest France|       1601|
|     Central Italy|       1530|
|    Southern Italy|       1439|
+------------------+-----------+
only showing top 20 rows



- O país que apresenta maior produção de vinhos é os Estados Unidos
- A região que apresenta a maior produção de vinhos é a Califórnia

**Como o Brasil está posicionado nesse ranking?**

In [9]:
result = spark.sql("select country as pais, count(country) as soma_pais \
from winemag_data \
where country = 'Brazil' \
group by country")

result.show()

+------+---------+
|  pais|soma_pais|
+------+---------+
|Brazil|       25|
+------+---------+



- O Brasil possui um total de 25 vinhos avaliados pela revista durante o período analisado

**Quais regiões brasileiras possuem vinhos avaliados?**

In [10]:
result = spark.sql("select province as regiao, count(province) as soma_regiao \
                    from winemag_data \
                    where country = 'Brazil' \
                    group by country, province, region_1, region_2 \
                    order by soma_regiao desc")
result.show()

+-----------------+-----------+
|           regiao|soma_regiao|
+-----------------+-----------+
|Vale dos Vinhedos|         11|
|           Brazil|          6|
|     Serra Gaúcha|          4|
|   Santa Catarina|          2|
| Serra do Sudeste|          1|
|    Vale Trentino|          1|
+-----------------+-----------+



Os vinhos produzidos no Brasil apareceram em 25 avaliações durante o período analisado.

A região brasileira que teve mais avaliações foi o Vale dos Vinhedos localizado na Serra Gaúcha no estado do Rio Grande do Sul com um total de 11 vinhos, porém há a presença de outros vinhos avaliados também localizados na Serra Gaúcha. Apesar de não haver total certeza de que o outros vinhos também não sejam especificamente da região do Vale dos Vinhedos, é evidenciado que a maior concentração de vinhos brasileiros avaliados pela Wine Enthusiast está na Serra Gaúcha.

Os achados nos levam a pensar que a Monte Venêto se localiza em uma região estratégica do ponto de vista de produção vinícula, visto que ela também se encontra na região da Serra Gaúcha. Isso chama a atenção para dois pontos: O primeiro se refere a competitividade que o seu novo produto deve apresentar em relação ao preço e aos vinhos que ja são produzidos na região. O segundo ponto está no fato de que a região já apresenta uma certa relevância internacional em relação a crítica especializada.

**Quais os tipos uvas são mais bem avaliadas?**

In [11]:
result = spark.sql("select country as pais, province as regiao, variety as tipo_uva, \
                    count(variety) as soma_vinhos \
                  from winemag_data \
                  group by country, province, variety \
                  order by soma_vinhos desc")
result.show()

+---------+----------------+--------------------+-----------+
|     pais|          regiao|            tipo_uva|soma_vinhos|
+---------+----------------+--------------------+-----------+
|       US|      California|          Pinot Noir|       7631|
|       US|      California|  Cabernet Sauvignon|       7444|
|       US|      California|          Chardonnay|       6535|
|   France|        Bordeaux|Bordeaux-style Re...|       4793|
|       US|      California|           Zinfandel|       3689|
|       US|      California|               Syrah|       2705|
|       US|          Oregon|          Pinot Noir|       2552|
|    Italy|         Tuscany|          Sangiovese|       2532|
|   France|        Burgundy|          Chardonnay|       2401|
|       US|      California|              Merlot|       2293|
|       US|      California|     Sauvignon Blanc|       2220|
|    Italy|         Tuscany|           Red Blend|       2215|
|    Italy|        Piedmont|            Nebbiolo|       2181|
|Argenti

In [12]:
result = spark.sql("select variety as tipo_uva, \
                    count(variety) as contagem_vinhos, \
                    round(avg(points), 0) as media_pontuacao, \
                    max(points) as max_pontuacao, \
                    min(points) as min_pontuacao \
        from winemag_data \
        group by variety \
        order by media_pontuacao desc")
result.show()

+--------------------+---------------+---------------+-------------+-------------+
|            tipo_uva|contagem_vinhos|media_pontuacao|max_pontuacao|min_pontuacao|
+--------------------+---------------+---------------+-------------+-------------+
|     Cabernet-Shiraz|              1|           96.0|           96|           96|
|               Tokay|             17|           93.0|           98|           86|
|            Muscadel|             10|           93.0|           97|           88|
|      Trousseau Gris|              1|           93.0|           93|           93|
|       Blauburgunder|              1|           93.0|           93|           93|
|            Roviello|              2|           93.0|           93|           92|
|              Sirica|              2|           93.0|           94|           92|
|      Loin de l'Oeil|              4|           92.0|           92|           91|
|  Provence red blend|             25|           92.0|           95|           87|
|   

- Os vinhos fabricados a partir da uva Pinot Noir foram os mais avaliados, porém a maior média de avaliação foi para os vinhos que possuem o blend Cabernet-Shiraz

**Quais uvas são mais bem avaliadas no Brasil?**

In [13]:
result = spark.sql("select province as regiao, variety as tipo_uva, \
                    count(variety) as contagem_vinhos, \
                    round(avg(points), 0) as media_pontuacao, \
                    max(points) as max_pontuacao, \
                    min(points) as min_pontuacao \
         from winemag_data \
         where country = 'Brazil' \
         group by province, variety \
         order by media_pontuacao desc")
result.show()

+-----------------+--------------------+---------------+---------------+-------------+-------------+
|           regiao|            tipo_uva|contagem_vinhos|media_pontuacao|max_pontuacao|min_pontuacao|
+-----------------+--------------------+---------------+---------------+-------------+-------------+
|     Serra Gaúcha|              Merlot|              1|           88.0|           88|           88|
|   Santa Catarina|Bordeaux-style Re...|              1|           86.0|           86|           86|
|           Brazil|           Red Blend|              2|           84.0|           84|           84|
|Vale dos Vinhedos|Cabernet Sauvigno...|              5|           84.0|           85|           84|
|           Brazil|     Sparkling Blend|              3|           83.0|           84|           82|
|   Santa Catarina|     Sparkling Blend|              1|           83.0|           83|           83|
|     Serra Gaúcha|Cabernet Sauvigno...|              3|           83.0|           83|     

- No Brasil os vinhos fabricados a partir da uva Merlot da Serra Gaúcha possuem uma maior média de avaliação com 88 pontos, seguido pelo Bordeaux-style Red Blend de Santa Catarina com 86 pontos. Devido às características de avaliação e os produtos da Monte Venêto a uva Bordeaux-style Red Blend merece destaque

**Qual a média de preço dos vinhos?**

- Média de preço geral (dólares) em relação ao tipo de uva

In [14]:
result = spark.sql("select distinct variety as tipo_uva, \
                    count(variety) as contagem_vinhos, \
                    round(avg(price), 0) as media_preco, \
                    round(max(price), 0) as max_preco, \
                    round(min(price), 0) as min_preco \
        from winemag_data \
        group by tipo_uva \
        order by media_preco desc")
result.show()

+--------------------+---------------+-----------+---------+---------+
|            tipo_uva|contagem_vinhos|media_preco|max_preco|min_preco|
+--------------------+---------------+-----------+---------+---------+
|     Cabernet-Shiraz|              1|      150.0|    150.0|    150.0|
|            Muscadel|             10|      141.0|     33.0|     11.0|
|             Mazuelo|              2|       99.0|     92.0|    105.0|
|                null|              0|       88.0|     88.0|     88.0|
|              Sonoma|              1|       87.0|     87.0|     87.0|
|          Tinto Fino|             75|       83.0|     90.0|     10.0|
|             Mission|              7|       83.0|     18.0|    120.0|
|               Tokay|             17|       82.0|     97.0|    120.0|
|      Carignan-Syrah|              1|       80.0|     80.0|     80.0|
|     Champagne Blend|           1238|       79.0|     99.0|     10.0|
|             Picolit|             29|       72.0|     90.0|    100.0|
|     

**Média de preços no Brasil em relação ao tipo de uva**

In [15]:
result = spark.sql("select distinct variety as tipo_uva, \
                    province as regiao, \
                    count(variety) as contagem_vinhos, \
                    round(avg(price), 0) as media_preco_dolar, \
                    round(avg(price * 5.14), 0) as media_preco_reais, \
                    round(min(price), 0) as min_preco_dolar, \
                    round(min(price * 5.14), 0) as min_preco_reais, \
                    round(max(price), 0) as max_preco_dolar, \
                    round(max(price * 5.14), 0) as max_preco_reais \
       from winemag_data \
       where country = 'Brazil' \
       group by province, tipo_uva \
       order by media_preco_dolar desc")
result.show()

+--------------------+-----------------+---------------+-----------------+-----------------+---------------+---------------+---------------+---------------+
|            tipo_uva|           regiao|contagem_vinhos|media_preco_dolar|media_preco_reais|min_preco_dolar|min_preco_reais|max_preco_dolar|max_preco_reais|
+--------------------+-----------------+---------------+-----------------+-----------------+---------------+---------------+---------------+---------------+
|Bordeaux-style Re...|   Santa Catarina|              1|             35.0|            180.0|           35.0|          180.0|           35.0|          180.0|
|     Sparkling Blend|   Santa Catarina|              1|             31.0|            159.0|           31.0|          159.0|           31.0|          159.0|
|           Red Blend|           Brazil|              2|             29.0|            149.0|           29.0|          149.0|           29.0|          149.0|
|Cabernet Sauvigno...|     Serra Gaúcha|              3|  

**Relacionando a variedade da uva e o preço do vinho**

- A região onde se localiza a Monte Venêto é estratégica do ponto de vista geográfico em termos de relevância no cenário de produção vinícula. Isso chama a atenção também para a necessidade de poricionar o seu produto em relação ao preço dos outros vinhos vendidos na região.

- As uvas mais bem avaliadas na região é a Merlot e a Bordeaux-style Red Blend. Essa última uva se apresenta como ideal para o lançamento do vinho, uma vez que já e uma uva cultivada pela empresa.

Agora iremos comparar os preços dos vinhos na região da Serra Gaúcha e vale dos vinhedos para entender melhor qual seria um valor interessante para vender o vinho.

In [16]:
result = spark.sql("select province as regiao, variety as tipo_uva, \
                    count(variety) as qtd_vinhos, \
                    winery as adega, designation as vinha, \
                    points as pontuacao, price as preco, \
                    round((price * 5.14), 0) as preco_reais \
         from winemag_data \
         where province = 'Serra Gaúcha' or province = 'Vale dos Vinhedos' \
         group by province, variety, winery, designation, points, price \
         order by preco desc;")
result.show()

+-----------------+--------------------+----------+--------------+------------------+---------+-----+-----------+
|           regiao|            tipo_uva|qtd_vinhos|         adega|             vinha|pontuacao|preco|preco_reais|
+-----------------+--------------------+----------+--------------+------------------+---------+-----+-----------+
|Vale dos Vinhedos|Cabernet Sauvigno...|         2|Vinícola Miolo|           Lote 43|       85| 35.0|      180.0|
|     Serra Gaúcha|Cabernet Sauvigno...|         3|Vinícola Miolo|RAR Family Reserve|       83| 27.0|      139.0|
|     Serra Gaúcha|              Merlot|         1| Lidio Carraro|             Agnus|       88| 15.0|       77.0|
|Vale dos Vinhedos|              Merlot|         3|Vinícola Miolo|              null|       82| 13.0|       67.0|
|Vale dos Vinhedos|  Cabernet Sauvignon|         3|Vinícola Miolo|              null|       81| 13.0|       67.0|
|Vale dos Vinhedos|Cabernet Sauvigno...|         3|Vinícola Miolo|    Cuvée Giuseppe|   

In [17]:
result = spark.sql("select province, count(variety) as qtd_vinhos \
                   from winemag_data \
                   where province = 'Serra Gaúcha' or province = 'Vale dos Vinhedos' \
                   group by province \
                   order by qtd_vinhos desc")
result.show()

+-----------------+----------+
|         province|qtd_vinhos|
+-----------------+----------+
|Vale dos Vinhedos|        11|
|     Serra Gaúcha|         4|
+-----------------+----------+



- A região do Vale dos Vinhedos apresentam uma maior quantidade de vinhos avaliados pela revista

In [18]:
result = spark.sql("select province as regiao, variety as tipo_uva, \
                    count(variety) as qtd_vinhos, \
                    winery as adega, designation as vinha, \
                    points as pontuacao, price as preco, \
                    round((price * 5.14), 0) as preco_reais \
                    from winemag_data \
                    where province = 'Serra Gaúcha' \
        group by province, variety, winery, designation, points, price \
        order by preco desc;")
result.show()

+------------+--------------------+----------+--------------+------------------+---------+-----+-----------+
|      regiao|            tipo_uva|qtd_vinhos|         adega|             vinha|pontuacao|preco|preco_reais|
+------------+--------------------+----------+--------------+------------------+---------+-----+-----------+
|Serra Gaúcha|Cabernet Sauvigno...|         3|Vinícola Miolo|RAR Family Reserve|       83| 27.0|      139.0|
|Serra Gaúcha|              Merlot|         1| Lidio Carraro|             Agnus|       88| 15.0|       77.0|
+------------+--------------------+----------+--------------+------------------+---------+-----+-----------+



- No total foram avaliados 15 vinhos originados da Serra Gaúcha.

- Os vinhos são produzidos a partir de uvas Merlot, Cabernet Sauvignon ou ainda um blend entre as duas uvas.

- A faixa de preço que compreende os vinhos varia entre 57 e 180 (reais), enquanto que a maioria dos vinhos avaliados (60%) estão na faixa entre 57 e 67 (reais)

In [19]:
result = spark.sql("select province as regiao, variety as tipo_uva, \
                    points as pontuacao, price as preco, \
                    round((price * 5.14), 0) as preco_reais \
         from winemag_data \
         where province = 'Serra Gaúcha' or province = 'Vale dos Vinhedos' \
         group by province, variety, winery, designation, points, price \
         order by preco desc;")
result.show()

+-----------------+--------------------+---------+-----+-----------+
|           regiao|            tipo_uva|pontuacao|preco|preco_reais|
+-----------------+--------------------+---------+-----+-----------+
|Vale dos Vinhedos|Cabernet Sauvigno...|       85| 35.0|      180.0|
|     Serra Gaúcha|Cabernet Sauvigno...|       83| 27.0|      139.0|
|     Serra Gaúcha|              Merlot|       88| 15.0|       77.0|
|Vale dos Vinhedos|              Merlot|       82| 13.0|       67.0|
|Vale dos Vinhedos|  Cabernet Sauvignon|       81| 13.0|       67.0|
|Vale dos Vinhedos|Cabernet Sauvigno...|       84| 11.0|       57.0|
+-----------------+--------------------+---------+-----+-----------+



**Conclusão**

A Monte Venêto poderia investir em um vinho que fosse fabricado a partir da uva Bordeaux-style Red Blend, uma vez que essa variedade já é cultivada pela empresa. A faixa de preço competitiva para o vinho seria entre 57 e 67 (reais). Um vinho pensado inicialmente a partir dessas informações pode torna-lo competitivo tanto em termos de preço como também em relação ao seu tipo, uma vez que vinhos da variedade Bordeaux-style Red Blend não foram avaliados na região e isso pode apontar para uma lacuna no mercado em relação a essa variedade de vinho.